# Phase 1 - Project 1

## Data Imports

In [ ]:
import csv    #for the cvs and txv files
import sqlite3 as sql   #for the db file
import pandas as pd    #Help visualize the data
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

## Read All files
Sample the files to see the contents


In [ ]:
df_1 = pd.read_csv("data/tmdb.movies.csv.gz")    #Main use, pull best and worst genre based on rating
df_1.head()

In [ ]:
df_2 = pd.read_csv("data/tn.movie_budgets.csv.gz")
df_2.head()

https://docs.python.org/3/library/codecs.html#standard-encodings - Helped with finding valid encoding 

In [ ]:
df_3 = pd.read_csv('data/rt.reviews.tsv.gz',delimiter = '\t', encoding = 'cp437' )
df_3.head()

In [ ]:
df_4 = pd.read_csv('data/rt.movie_info.tsv.gz',delimiter = '\t')
df_4.head()

In [ ]:
df_5 = pd.read_csv('data/bom.movie_gross.csv.gz',delimiter = '\t')
df_5.head()

In [ ]:
conn = sql.connect('data/im.db/im.db')

#pd.read_sql('SELECT * FROM  zipfile(data/im.db.zip)', conn)   #https://www.sqlite.org/zipfile.html

In [ ]:
pd.read_sql("""SELECT * FROM persons""", conn)

## Data Cleaning

### The Movie DB 
Useful keys: Popularity, vote count, vote average

#### Data Exploration

In [ ]:
print(df_1.columns)
df_1['genre_ids']    #What does these numbers mean? - 
df_1['id']    #Each movie has a unique id? 
df_1['original_language']    #Only english movies? - No, there are multiples
df_1['original_title']     #Title of the movie
df_1['popularity']    #Ranking out of ... 100 , Website looks like percentage - based on amount timed watched?
df_1.sort_values('popularity').tail()
#Release data, title are pretty obvious, may need to look in title vs original title
df_1.loc[df_1['vote_average'] == 0]  #Quite a few with not votes and therefore no vote average
#df_1['vote_average'].plot(kind = 'hist')    #- Suject to remove with no votes


#### Data Filtering

In [ ]:
####   Data Cleaning    ###
print(df_1.info())    #Unknown column
df_1['Unnamed: 0']    #No use, index column
df_1.drop("Unnamed: 0", axis = 1, inplace= True)   #Remove the column
df_1.columns    #Confirm the drop

In [ ]:
###    Check missing values    ###
print (df_1.isna().sum())    #No NaN values
print (df_1.isnull().sum())    #No Null Values


In [ ]:
###    Remove dups    ###
df_1.duplicated().sum()    #There are duplicated movies? Multiple languages?
df_1[df_1.duplicated(keep = False)].sort_values(by = 'id')    #Looks like straight copies - Need to remove
df_1.drop_duplicates(inplace = True)    #Remove dups
df_1.duplicated().sum()   #Check again for dups


#### Data Visualization
Important columns: popularity and genre


In [ ]:
#Sort by popularity to see whats type of movies are the best
df_1.sort_values('popularity', inplace = True,ascending= False)

df_1['genre_ids'] = ((df_1['genre_ids'].map(lambda x: x.strip('[]'))).map(lambda x: x.replace(' ', ''))).map(lambda x: x.split(','))
df_1['genre_ids'][0]
df1_top_movies = df_1[0:100]
df1_worst_movie = df_1[-100:]

##### Graphs

In [ ]:
# Convert the genre id to names for understanding purposes
convert = {'28' : 'Action', '12': 'Adventure', '35': 'Comedy', '99': 'Documentary', 
           '18': 'Drama', '14': 'Fantasy', '27': 'Horror', '878' : 'Sci-Fi', '53' : 'Thriller',
           '16': 'Animation', '80' : 'Crime', '10751' : 'Family','14':'Fantasy',
           "36" : 'History','27': 'Horror', '10402':'Music', '9648':'Mystery',
           '10749':'Romance','10770': 'TV Movie','10752': 'War','37':'Western', '': "NULL" }

# Function count all the genre and group them
def count_genre(df_list):
    genre_count = {}    #https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee

    for genre in df_list['genre_ids']:
        for ids in genre:
            if(genre_count.get(convert[ids])):
                genre_count[convert[ids]] += 1
            else:
                genre_count[convert[ids]] = 1
    return sorted(genre_count.items(), key = lambda x: x[1], reverse=True)
   
df1_top_genre = count_genre(df1_top_movies)    #Top genre from the most popular movies
df1_worst_genre = count_genre(df1_worst_movie)    #Top genre from teh worst movies
x, y = zip(*df1_top_genre)    #Seperate the tuple into movie and count
x2,y2 = zip(*df1_worst_genre)


fig, ax = plt.subplots(1,2, figsize = (20,10))
colors1 = ['red' if genre in x2[0:5] else 'green' for genre in x]    #Red for genre in the worst genre list
colors2 = ['cyan' if genre in x[0:5] else 'black' for genre in x2]    #Cyan for genre in the top genre list

ax[0].barh(x,y, color = colors1);
ax[0].set_xlabel("Frequency");
ax[0].set_ylabel("Genre");
ax[0].set_title("Best Genre from Top 100 Movies");
ax[0].legend(["Good Tags"]);

ax[1].barh(x2,y2, color = colors2);
ax[1].set_xlabel("Frequency");
ax[1].set_ylabel("Genre");
ax[1].set_title("Worst Genre from Worst 100 Movies");
ax[1].legend(["Bad Tags"]);

### The Number

#### Data Exploration

In [ ]:
df_2.duplicated().sum()    #No dups?
df_2.isna().sum()    #No NaN
df_2.isnull().sum()    #No Null
df_2.info()    #All usable except release data right now

#### Data Filtering

In [ ]:
###    Change the columns to integer for comparing and statistical methods    ###

df_2['production_budget'] = df_2['production_budget'].map(lambda x: x.strip('$'))    #Take out the $
df_2['production_budget'] = df_2['production_budget'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['production_budget'] = df_2['production_budget'].astype('int64')    #Convert to int64

df_2['domestic_gross'] = df_2['domestic_gross'].map(lambda x: x.strip('$'))    #Take out the $
df_2['domestic_gross'] = df_2['domestic_gross'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['domestic_gross'] = df_2['domestic_gross'].astype('int64')

df_2['worldwide_gross'] = df_2['worldwide_gross'].map(lambda x: x.strip('$'))   #Take out the $
df_2['worldwide_gross'] = df_2['worldwide_gross'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['worldwide_gross'] = df_2['worldwide_gross'].astype('int64')

In [ ]:
###   Make the ratio for both the profit
df_2['prod_dom_ratio'] = df_2['domestic_gross'] / df_2['production_budget']    #Ratio on the more proftitable movie - Domestic
df_2['prod_world_ratio'] = df_2['worldwide_gross'] / df_2['production_budget']    #Ratio on the more proftitable movie - worldwide
df_2.sort_values(['prod_dom_ratio'], ascending= False, inplace= True)
df_2.head(10)     #One outlier...I see
###   Domestic to Budget    ###
print((df_2['prod_dom_ratio'] > 1).sum())    #Half the movie broke even - Categorical?
print((df_2['prod_dom_ratio'] > 2).sum())    #1/4 of the movie doubled <- Goal
print((df_2['prod_dom_ratio'] > 3).sum())    #Best case for most movies 
###   World to Budget    ###
df_2.sort_values(['worldwide_gross'], ascending= False, inplace= True)
print((df_2['prod_world_ratio'] > 1).sum()) 
print((df_2['prod_world_ratio'] > 2).sum())
print((df_2['prod_world_ratio'] > 3).sum())   

##### Ratio for domestic gross 

In [ ]:
# Titles from the first dataset
df1_top_list = list(df1_top_movies['title'])    #Top 100 movies from the first dataset
df1_top_list


In [172]:
###    Production vs Domestic Ratio    ###
df_2.sort_values(['prod_dom_ratio'], ascending= False, inplace= True)
df2_most_prof = df_2['movie'].loc[df_2['prod_dom_ratio'] > 1]
df2_most_prof_2 = df_2['movie'].loc[df_2['prod_dom_ratio'] > 2]
df2_most_prof_3 = df_2['movie'].loc[df_2['prod_dom_ratio'] > 3] 
###    Production vs WorldWide Ratio    ###
df_2.sort_values(['prod_world_ratio'], ascending= False, inplace= True)
df2_most_prof_world = df_2['movie'].loc[df_2['prod_world_ratio'] > 1]  
df2_most_prof_2_world = df_2['movie'].loc[df_2['prod_world_ratio'] > 2]  
df2_most_prof_3_world = df_2['movie'].loc[df_2['prod_world_ratio'] > 3]  
print(len(df_2['movie'].loc[(df_2['prod_world_ratio'] <1) & (df_2['production_budget'] < 17000000)]))
print(len(df_2['movie'].loc[(df_2['prod_world_ratio'] <1)]))
df_2

1297
2125


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,prod_dom_ratio,prod_world_ratio
5745,46,"Jun 30, 1972",Deep Throat,25000,45000000,45000000,1800.000000,1800.000000
5613,14,"Mar 21, 1980",Mad Max,200000,8750000,99750000,43.750000,498.750000
5492,93,"Sep 25, 2009",Paranormal Activity,450000,107918810,194183034,239.819578,431.517853
5679,80,"Jul 10, 2015",The Gallows,100000,22764410,41656474,227.644100,416.564740
5406,7,"Jul 14, 1999",The Blair Witch Project,600000,140539099,248300000,234.231832,413.833333
...,...,...,...,...,...,...,...,...
5757,58,"Dec 1, 2003",Steel Spirit,20000,0,0,0.000000,0.000000
5698,99,"Jul 7, 2015",Tiger Orange,100000,0,0,0.000000,0.000000
1207,8,"Nov 22, 2019",The Rhythm Section,50000000,0,0,0.000000,0.000000
4990,91,"Jul 9, 2013",Coffee Town,2000000,0,0,0.000000,0.000000


In [ ]:
###    Function that takes the data frame, takes the movie name if they are also in the top movie list    ###
def list_of_movies(df_list):
    movie_list = []    #Make a empty list
    for name in df_list:    #For each movie in the list    
        for top in df1_top_list:    #Check if any of the most popular list are in the second list
            if(name == top):    #If match
                movie_list.append(name)     #Add the list
    return movie_list            

In [ ]:
###    Over_# is the list of genre based on the movies with the respective ratios
###    If the movie is also in the most profitable list and the most popular list,
###    they count the genre for that ratio
over_1 = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof))])
over_2 = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof_2))])
over_3 = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof_3))])

over_1_w = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof_world))])
over_2_w = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof_2_world))])
over_3_w = count_genre(df_1.loc[df_1['title'].isin(list_of_movies(df2_most_prof_3_world))])

all_genre = count_genre(df_1)


##### Data Comparison - Ratio of 3 and up

In [ ]:
ax = plt.subplot()
x, y = zip(*all_genre)


#fig.suptitle('Domestic Gross Data', fontsize=15)
ax.barh(x,y, color = 'red');
ax.set_title('Most Popular Genre - All Movies');
ax.set_xlabel('Frequency');
ax.set_ylabel('Genre');

In [ ]:
fig , ax = plt.subplots(2,2, figsize =(15, 10))
x, y = zip(*over_1)
x2, y2 = zip(*over_2)
x3, y3 = zip(*over_3)

fig.suptitle('Domestic Gross Data', fontsize=15)
ax[0,0].barh(x,y, color = 'red', label = 'Over 1')
ax[0,0].legend();
ax[0,0].set_title('Most Popular Genre Based on Ratio: >1');
ax[0,0].set_xlabel('Frequency')
ax[0,0].set_ylabel('Genre')
ax[0,0].set_xlim([0,45])

ax[0,1].barh(x2,y2, color = 'blue',label = 'Over 2')
ax[0,1].legend();
ax[0,1].set_title('Most Popular Genre Based on Ratio: >2');
ax[0,1].set_xlabel('Frequency')
ax[0,1].set_ylabel('Genre')
ax[0,1].set_xlim([0,45])

ax[1,0].barh(x3,y3, color = 'green', label = 'Over 3')
ax[1,0].legend();
ax[1,0].set_title('Most Popular Genre Based on Ratio: >3');
ax[1,0].set_xlabel('Frequency')
ax[1,0].set_ylabel('Genre')
ax[1,0].set_xlim([0,45])

ax[1,1].barh(x,y, color = 'red', label = 'Over 1');
ax[1,1].barh(x2,y2,  color = 'blue', label = 'Over 2');
ax[1,1].barh(x3,y3,  color = 'green', label = 'Over 3');
ax[1,1].legend();
ax[1,1].set_title('Most Popular Genre Based on Ratio');
ax[1,1].set_xlabel('Frequency')
ax[1,1].set_ylabel('Genre')
ax[1,1].set_xlim([0,45])

In [ ]:
ax = plt.subplot()
x, y = zip(*over_1)
x2, y2 = zip(*over_2)
x3, y3 = zip(*over_3)
ax.barh(x,y, color = 'red', label = 'Over 1');
ax.barh(x2,y2,  color = 'blue', label = 'Over 2');
ax.barh(x3,y3,  color = 'green', label = 'Over 3');
ax.legend();
ax.set_xlabel('Frequency')
ax.set_ylabel('Genre')
ax.set_title('Most Popular Genre Based on Ratio');
ax.set_xlim([0,45])

In [ ]:
fig2 , ax2 = plt.subplots(2,2, figsize =(15, 10))
x4, y4 = zip(*over_1_w)
x5, y5 = zip(*over_2_w)
x6, y6 = zip(*over_3_w)

fig2.suptitle('Foreign Gross Data', fontsize=15)
ax2[0,0].barh(x4,y4, color = 'red', label = 'Over 1')
ax2[0,0].legend();
ax2[0,0].set_title('Most Popular Genre Based on Ratio: >1');
ax2[0,0].set_xlabel('Frequency')
ax2[0,0].set_ylabel('Genre')
ax2[0,0].set_xlim([0,60])

ax2[0,1].barh(x5,y5, color = 'blue',label = 'Over 2')
ax2[0,1].legend();
ax2[0,1].set_title('Most Popular Genre Based on Ratio: >2');
ax2[0,1].set_xlabel('Frequency')
ax2[0,1].set_ylabel('Genre')
ax2[0,1].set_xlim([0,60])

ax2[1,0].barh(x6,y6, color = 'green', label = 'Over 3')
ax2[1,0].legend();
ax2[1,0].set_title('Most Popular Genre Based on Ratio: >3');
ax2[1,0].set_xlabel('Frequency')
ax2[1,0].set_ylabel('Genre')
ax2[1,0].set_xlim([0,60])

ax2[1,1].barh(x4,y4, color = 'red', label = 'Over 1');
ax2[1,1].barh(x5,y5,  color = 'blue', label = 'Over 2');
ax2[1,1].barh(x6,y6,  color = 'green', label = 'Over 3');
ax2[1,1].legend();
ax2[1,1].set_title('Most Popular Genre Based on Ratio');
ax2[1,1].set_xlim([0,60])

In [ ]:
print("Highest Worldwide Grossing movies all have higher than 3 ratio?", over_1_w == over_2_w == over_3_w)
print("""Number of movies with a ratio of 1 and above? 
      Domestic: {}, Foreign: {} """.format(len(df2_most_prof),len(df2_most_prof_world)))
print("""Number of movies with a ratio of 2 and above? 
      Domestic: {}, Foreign: {} """.format(len(df2_most_prof_2),len(df2_most_prof_2_world)))
print("""Number of movies with a ratio of 3 and above? 
      Domestic: {}, Foreign: {} """.format(len(df2_most_prof_3),len(df2_most_prof_3_world)))
print("""Number of movies with a ratio of under 1? 
      Domestic: {}, Foreign: {} """.format(len(df_2['movie'].loc[df_2['prod_dom_ratio'] <= 1]),len(df_2['movie'].loc[df_2['prod_world_ratio'] <= 1])))


In [ ]:
df_2.sort_values(['production_budget'], ascending= False, inplace= True)
IQR = df_2['prod_world_ratio'].quantile(.75) - df_2['prod_world_ratio'].quantile(.25)
min_out = df_2['prod_world_ratio'].median() - IQR*1.5
max_out = df_2['prod_world_ratio'].median() + IQR*1.5
max_out
outlier_removed = df_2.loc[df_2['prod_world_ratio'] < 6.607385548232989]
outlier_removed['prod_world_ratio'].describe()
scat_x = outlier_removed['production_budget'].loc[outlier_removed['prod_dom_ratio'] > 1]
scat_y = outlier_removed['prod_dom_ratio'].loc[outlier_removed['prod_dom_ratio'] > 1]
fig, ax = plt.subplots()
ax.scatter(scat_x, scat_y , alpha= .2, color = 'green')
#line = plt.axvline(200000000, color = 'red');
ax.set_xlabel('Production Budget')
ax.set_xticks([0,100000000,200000000]);
ax.set_xlim(0,100000000)
x = list(range(0,74000000,1000))
y = [6-x2/15000000 for x2 in x]
ax.axhline(1.7083098257683216 , color = 'blue', label = 'Median ratio');
ax.axvline(17000000.0,color = 'cyan', label = 'Median Budget')
ax.legend()
ax.plot(x, y, color = 'red', alpha = .5)

In [ ]:
IQR = df_2['prod_world_ratio'].quantile(.75) - df_2['prod_world_ratio'].quantile(.25)
min_out = df_2['prod_world_ratio'].median() - IQR*1.5
max_out = df_2['prod_world_ratio'].median() + IQR*1.5
df_2['prod_world_ratio'].describe()
df_2['production_budget'].median()
df_2['production_budget'].quantile(.75)

##### Data Comparison - Ratio of 2 and upmedian

#### Data Visualization

### Rotten Tomatoes

Id is probably Movie ID 
Review are the written review of the critic
Rating is out of 5
Freshness appears to be 3>Fresh and lower is rotten
Top critics are acclaimed critics on rottom tomatoes
Publisher and Release data have no use at the moment


In [ ]:
df_3.head()
#df_3.info()
#df_3.isna().sum()
df_3.loc[df_3['review'].isna()]
df3_top_critic = df_3.loc[df_3['top_critic'] == 1]
print(len(df3_top_critic['critic'].unique()))    #856 unique critics
print(len(df3_top_critic))    #13096 review from top critics


In [ ]:
df3_count_review = {}
df3_top_critic.head()


In [ ]:
df_4.head()
df_4.sort_values('box_office')    #Not alot of info like the other data set
df_4.isna().sum()    #1560 entires, 1220 are NaN. Data set not usable for current project.
df_4.info()

In [ ]:
df_5

In [ ]:
pd.read_sql("""SELECT  persons.primary_name, AVG(averagerating), COUNT(movie_id)
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)

GROUP BY persons.primary_name
HAVING AVG(averagerating) > 6.332729
ORDER BY AVG(averagerating) DESC
""", conn)

In [ ]:
pd.read_sql("""SELECT  primary_name, COUNT(movie_id)
FROM movie_ratings

""", conn)

In [ ]:
pd.read_sql("""SELECT persons.primary_name, COUNT(movie_id), AVG(averagerating)
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)
GROUP BY persons.primary_name
HAVING AVG(averagerating) > 6.332729 AND COUNT(movie_id) > 10
ORDER BY AVG(averagerating) DESC
""", conn)

In [ ]:
181387 movies
73103 movies group by movie Id
57241 movie group by person id

In [ ]:
pd.read_sql("""SELECT persons.primary_name, COUNT(movie_id), AVG(averagerating)
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)
GROUP BY persons.primary_name

""", conn)

In [ ]:
list_of_directors= pd.read_sql("""SELECT primary_title, persons.primary_name, COUNT(movie_id), AVG(averagerating)
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)
GROUP BY primary_title
HAVING AVG(averagerating) > 6.332729 AND COUNT(movie_id) > 10

ORDER BY AVG(averagerating) DESC
LIMIT 100
""", conn)

In [ ]:
list(list_of_directors['primary_title'])

In [ ]:
best_dir_movies = []
for movie in list(list_of_directors['primary_title']):
    if movie in df1_top_list:
        best_dir_movies.append(movie)
best_dir_movies

In [174]:
Recommended_dir = pd.read_sql("""SELECT primary_title AS 'Movie Title'
                ,persons.primary_name AS 'Director Name'
                , COUNT(movie_id) AS 'Number of Movies' 
                ,AVG(averagerating) AS 'Average Movie Ratings'
FROM movie_ratings
JOIN movie_basics
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING (person_id)
WHERE primary_title IN ('Avengers: Infinity War',
                         'How to Train Your Dragon',
                         'Guardians of the Galaxy',
                         'Zootopia',
                         'Captain America: Civil War',
                         'Big Hero 6',
                         'Guardians of the Galaxy Vol. 2')
GROUP BY primary_title
HAVING AVG(averagerating) > 6.332729 AND COUNT(movie_id) > 10

ORDER BY AVG(averagerating) DESC
LIMIT 100
""", conn)

In [173]:
set(Recommended_dir['Director Name'])

{'Anthony Russo',
 'Byron Howard',
 'Chris Williams',
 'Dean DeBlois',
 'James Gunn'}

In [175]:
Recommended_dir

,Movie Title,Director Name,Number of Movies,Average Movie Ratings
0,Avengers: Infinity War,Anthony Russo,24,8.5
1,How to Train Your Dragon,Dean DeBlois,12,8.1
2,Guardians of the Galaxy,James Gunn,11,8.1
3,Zootopia,Byron Howard,24,8.0
4,Captain America: Civil War,Anthony Russo,12,7.8
5,Big Hero 6,Chris Williams,12,7.8
6,Guardians of the Galaxy Vol. 2,James Gunn,13,7.7
